## Notebook settings

In [1]:
%load_ext autoreload
%autoreload 2

%autosave 10

%load_ext lab_black

Autosaving every 10 seconds


## Imports

In [2]:
from lfp_analysis.data import *

In [3]:
import numpy as np
import pandas as pd
import h5py
from librosa.util import frame

from pathlib import Path

import matplotlib.pyplot as plt

%matplotlib widget

# Label processor:

In [4]:
def make_df(label, win_len_sec, perc_valid=0.8):
    win_len_samp = np.int(win_len_sec * 2048)
    n_win = len(label) // win_len_samp

    # Window idx:
    id_l = np.array(range(n_win)) * win_len_samp
    data_df = pd.DataFrame(
        np.stack((id_l, id_l + win_len_samp)).T, columns=["id_start", "id_end"]
    )

    # Train / Valid split
    data_df["is_valid"] = 0
    data_df.iloc[int(n_win * perc_valid) :, 2] = 1

    # Get label for each window:
    data_df["label"] = np.any(frame(label, win_len_samp, win_len_samp), axis=0)

    return data_df

# Import data:

In [5]:
ENV = "ET1_Pegboard_OFF"
H5_SOURCE = Path(f"./data/preprocessed/{ENV}.h5")

In [6]:
h5 = h5py.File(H5_SOURCE, "r")
label = h5["label"][:]
h5.close()

In [7]:
label.shape

(1455948,)

In [8]:
df = make_df(label, 0.125, perc_valid=0.8)

# Save to file:

In [9]:
TARGET_FILE = Path(f"./data/TF_and_df/{ENV}.csv")

In [10]:
df.to_csv(TARGET_FILE)